In [3]:
from dotenv import load_dotenv
import os
from pathlib import Path
from datetime import datetime, timedelta

import torch
from googleapiclient.discovery import build
import yt_dlp
import whisper
import re
from tqdm import tqdm

import google.generativeai as genai
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

import pandas as pd
from pathlib import Path
import os
import shutil

C:\Users\kk200\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
folder_path = './preprocessed_data/llm/predict_text'
path_obj = Path(folder_path)

if not path_obj.is_dir():
    print(f"오류: '{folder_path}'는 유효한 폴더 경로가 아닙니다.")

print(f"\n'{folder_path}' 내의 모든 파일 목록 (pathlib 사용):")

dfs = []
for file_path in path_obj.rglob('*.csv'): 
    if file_path.is_file(): # 파일만 선택
        df = pd.read_csv(file_path, encoding='utf-8')
        dfs.append(df)
        
df_t = pd.concat(dfs)
df_t = df_t.sort_values(by=['code', 'year', 'month', 'week'], ascending=True, ignore_index=True)
df_t.to_csv('./preprocessed_data/llm/predict_total/predict_text.csv', encoding='utf-8', index=False)

In [18]:
folder_path = './preprocessed_data/llm/predict_video'
path_obj = Path(folder_path)

if not path_obj.is_dir():
    print(f"오류: '{folder_path}'는 유효한 폴더 경로가 아닙니다.")

print(f"\n'{folder_path}' 내의 모든 파일 목록 (pathlib 사용):")

dfs = []
for file_path in path_obj.rglob('*.csv'): 
    if file_path.is_file(): # 파일만 선택
        df = pd.read_csv(file_path, encoding='utf-8')
        dfs.append(df)
        
df_v = pd.concat(dfs)
df_v = df_v.sort_values(by='code', ascending=True, ignore_index=True)
df_v.to_csv('./preprocessed_data/llm/predict_total/predict_video.csv', encoding='utf-8', index=False)


'./preprocessed_data/llm/predict_video' 내의 모든 파일 목록 (pathlib 사용):


## 영상 자료가 없는 것 -> 기사로 대체 

In [26]:
df_v = pd.read_csv('./preprocessed_data/llm/predict_total/predict_video.csv', encoding='utf-8')
df_t = pd.read_csv('./preprocessed_data/llm/predict_total/predict_text.csv', encoding='utf-8')

nan_list = pd.isna(df_v['prediction'])
df_v[nan_list] = df_t[nan_list]
df_v.to_csv('./preprocessed_data/llm/predict_total/predict_mix.csv', encoding='utf-8', index=False)

## 영상 자료 없는 것 + 예측 불가 -> 기사로 대체

In [28]:
df_v = pd.read_csv('./preprocessed_data/llm/predict_total/predict_video.csv', encoding='utf-8')
df_t = pd.read_csv('./preprocessed_data/llm/predict_total/predict_text.csv', encoding='utf-8')

nan_list = pd.isna(df_v['prediction']) | (df_v['prediction'] == '불가능')
df_v[nan_list] = df_t[nan_list]
df_v.to_csv('./preprocessed_data/llm/predict_total/predict_mix.csv', encoding='utf-8', index=False)

,year,quarter,month,week,code,name,sector,upload_dt,prediction,reason,score
0,2022,Q1,3,4,120,CJ대한통운,산업재,NaN,NaN,NaN,NaN
1,2022,Q1,3,3,120,CJ대한통운,산업재,NaN,NaN,NaN,NaN
3,2022,Q1,3,1,120,CJ대한통운,산업재,2022-03-02,중립,요약본에서는 CJ대한통운과 관련된 전국택배 노조의 파업이 마무리되었다는 내용이 포함...,0.0
4,2021,Q2,6,4,120,CJ대한통운,산업재,NaN,NaN,NaN,NaN
5,2021,Q2,6,3,120,CJ대한통운,산업재,2021-06-17,중립,요약본에서는 CJ대한통운에 대한 구체적인 정보나 긍정적/부정적인 요소가 언급되지 않...,0.0
...,...,...,...,...,...,...,...,...,...,...,...
5707,2024,Q3,9,4,86280,현대글로비스,산업재,NaN,NaN,NaN,NaN
5708,2024,Q4,10,1,86280,현대글로비스,산업재,NaN,NaN,NaN,NaN
5709,2024,Q4,10,2,86280,현대글로비스,산업재,NaN,NaN,NaN,NaN
5710,2024,Q4,10,3,86280,현대글로비스,산업재,NaN,NaN,NaN,NaN


## 자료 수집 결측치 확인

In [7]:
df_v = pd.read_csv('./preprocessed_data/llm/predict_total/predict_video.csv', encoding='utf-8')
df_t = pd.read_csv('./preprocessed_data/llm/predict_total/predict_text.csv', encoding='utf-8')
df_m = pd.read_csv('./preprocessed_data/llm/predict_total/predict_mix.csv', encoding='utf-8')

nan_list = pd.isna(df_v['prediction'])
nan_num = sum(nan_list)
total_num = len(df_v)
print(f'video 결측: {nan_num} / {total_num}')


nan_list = pd.isna(df_t['prediction'])
nan_num = sum(nan_list)
total_num = len(df_t)
print(f'text 결측: {nan_num} / {total_num}')


nan_list = pd.isna(df_m['prediction'])
nan_num = sum(nan_list)
total_num = len(df_m)
print(f'video+text 결측: {nan_num} / {total_num}')

video 결측: 2046 / 5712
text 결측: 1710 / 5712
video+text 결측: 876 / 5712
